In [ ]:
pip install datasets trl bitsandbytes peft accelerate

In [ ]:
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from accelerate import Accelerator
import torch

In [ ]:
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training, AutoPeftModelForCausalLM

In [ ]:
from transformers import AutoModelForCausalLM , AutoTokenizer, BitsAndBytesConfig, TrainingArguments

In [ ]:
df = load_dataset("tatsu-lab/alpaca",split ="train")

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
data = df.to_pandas()
data = data.sample(1000)

In [ ]:
data['text'] = data[['instruction' ,'input','output']].apply(lambda x : "Human : " + x['instruction'] + " " + x["input"] + "\nAI:" + x['output'] ,axis = 1)

In [ ]:
df = Dataset.from_pandas(data)

In [ ]:
model ="tinyllama/tinyllama-1.1b-chat-v1.0"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model,device_map="auto")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
peft_model = get_peft_model(model,lora_config)

In [ ]:
 training_args = TrainingArguments(
     output_dir="./results",
     per_device_train_batch_size=8,
     gradient_accumulation_steps=1,
     optim="paged_adamw_32bit",
     learning_rate=2e-4,
     lr_scheduler_type="cosine",
     save_strategy="epoch",
     save_total_limit=3,
     logging_steps=10,
     num_train_epochs=1,
     max_steps=100,
     fp16 = True,
 )

In [ ]:
trainer  = SFTTrainer(
    model=peft_model,
    train_dataset=df,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=training_args,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,2.191500
20,1.929400
30,1.653200
40,1.596600
50,1.458400
60,1.437200
70,1.408000
80,1.450800
90,1.439700
100,1.505600


TrainOutput(global_step=100, training_loss=1.6070473861694337, metrics={'train_runtime': 100.2748, 'train_samples_per_second': 7.978, 'train_steps_per_second': 0.997, 'total_flos': 968960216530944.0, 'train_loss': 1.6070473861694337, 'epoch': 0.8})

In [ ]:
trainer.save_model("Finetuned_Model")
tokenizer.save_pretrained("Finetuned_tokeinizer")

('Finetuned_tokeinizer/tokenizer_config.json',
 'Finetuned_tokeinizer/special_tokens_map.json',
 'Finetuned_tokeinizer/tokenizer.model',
 'Finetuned_tokeinizer/added_tokens.json',
 'Finetuned_tokeinizer/tokenizer.json')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset,Dataset
from trl import DPOTrainer
from peft import get_peft_model,LoraConfig
from accelerate import Accelerator

In [ ]:
df = load_dataset("Anthropic/hh-rlhf",split="train")
df

README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})

In [ ]:
df = df.to_pandas()

In [ ]:
df['prompt'] = df['chosen'].apply(lambda x : x.split("Assistant: ")[0])
df['prompt'] = df['chosen'].apply(lambda x : "Assistant: " + x.split("Assistant: ")[-1])
df['rejected'] = df['rejected'].apply(lambda x : "Assistant: " + x.split("Assistant: ")[-1])
df = df.sample(1000)

In [ ]:
data = Dataset.from_pandas(df)

In [ ]:
pip install -U bitsandbytes

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/Finetuned_tokeinizer")

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "/content/Finetuned_Model",
                                             device_map = Accelerator().local_process_index,
                                             torch_dtype = torch.float16)

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
peft_model_ft = get_peft_model(model,lora_config)

In [ ]:
peft_model_ft.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset,Dataset
from trl import DPOTrainer, DPOConfig

In [ ]:
from trl import SFTConfig

In [ ]:
dpo_config = DPOConfig(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = DPOTrainer(
    model=peft_model_ft,
    processing_class=tokenizer,
    args=dpo_config,
    train_dataset=data,
    peft_config=lora_config,
)

Extracting prompt in train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
500,0.072200
1000,0.011300
1500,0.006100


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=1500, training_loss=0.029891847451527915, metrics={'train_runtime': 1693.5947, 'train_samples_per_second': 1.771, 'train_steps_per_second': 0.886, 'total_flos': 0.0, 'train_loss': 0.029891847451527915, 'epoch': 3.0})

In [ ]:
trainer.save_model("finetuned-dpo")
tokenizer.save_pretrained("finetuned-tokenizer_dpo")

('finetuned-tokenizer_dpo/tokenizer_config.json',
 'finetuned-tokenizer_dpo/special_tokens_map.json',
 'finetuned-tokenizer_dpo/tokenizer.model',
 'finetuned-tokenizer_dpo/added_tokens.json',
 'finetuned-tokenizer_dpo/tokenizer.json')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
original_tokenizer = AutoTokenizer.from_pretrained("tinyllama/tinyllama-1.1b-chat-v1.0")  # Original model
original_model = AutoModelForCausalLM.from_pretrained("tinyllama/tinyllama-1.1b-chat-v1.0",
                                                      torch_dtype=torch.float16,
                                                      device_map="auto")

In [ ]:
dpo_tokenizer = AutoTokenizer.from_pretrained("finetuned-tokenizer_dpo")  # DPO model
dpo_model = AutoModelForCausalLM.from_pretrained("finetuned-dpo",
                                                torch_dtype=torch.float16,
                                                device_map="auto")

In [ ]:
dpo_model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, bia

In [ ]:
original_generator = pipeline(task="text-generation",
                               model=original_model,
                               tokenizer=original_tokenizer)
dpo_generator = pipeline(task="text-generation",
                          model=dpo_model,
                          tokenizer=dpo_tokenizer)

Device set to use cuda:0
Device set to use cuda:0


In [ ]:
print(dpo_generator)

In [ ]:
dpo_generator = pipeline(
    task="text-generation",
    model=dpo_model,
    tokenizer=dpo_tokenizer,
    pad_token_id=dpo_tokenizer.eos_token_id
)

Device set to use cuda:0


In [ ]:
if dpo_tokenizer.pad_token is None:
    dpo_tokenizer.pad_token = dpo_tokenizer.eos_token

In [ ]:
prompt = "Translate this sentence into French: 'The sky is blue."

In [ ]:
inputs = dpo_tokenizer(prompt, return_tensors="pt").to("cuda")
output = dpo_model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    repetition_penalty=1.2
)

In [ ]:
generated_text = dpo_tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Translate this sentence into French: 'The sky is blue. Blue, a color,'"
Answer: "La lune est bleue. Bleu, un couleur,"


In [ ]:
prompt = "What are some polite ways to decline an Graduation?"

In [ ]:
inputs = dpo_tokenizer(prompt, return_tensors="pt").to("cuda")
output = dpo_model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    repetition_penalty=1.2
)

In [ ]:
generated_text = dpo_tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

What are some polite ways to decline an Graduation? I mean, we do not know each other that well and it might lead to awkwardness in the future. Do you think anyone would even notice?
> 
><|assistant|> Hi! Here's a response: As long as it is politely said and done before/after Graduation, I don't believe there will be any issues with awkwardness or anything else. It may cause people around them to feel uncomfortable but they can handle
